In [1]:
!pip install timm

In [2]:
import timm

In [3]:
print('전체 모델 개수', len(timm.list_models()))

전체 모델 개수 1167


In [4]:
print('전체 모델 개수', len(timm.list_models(pretrained=True)))

전체 모델 개수 1492


In [5]:
print('resnet으로 시작하는 pretrained 모델 개수', len(timm.list_models('resnet*', pretrained=True)))
print('resnet으로 시작하는 모델 개수', len(timm.list_models('resnet*')))

resnet으로 시작하는 pretrained 모델 개수 117
resnet으로 시작하는 모델 개수 75


## 피부암 진단 모델 고도화 학습

In [6]:
# ResNet50 모델 선언
model = timm.create_model('resnet50d', pretrained=True)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


model.safetensors:   0%|          | 0.00/103M [00:00<?, ?B/s]

In [7]:
# 흑백 영상을 입력 받는 모델 선언
model_grayscale = timm.create_model('resnet50d', in_chans=1, pretrained=True)

# 출력 클래스 수가 10개인 모델 선언
model_10cl = timm.create_model('resnet50d', num_classes=10, pretrained=True)

In [8]:
# 모델 구조 확인
print(model)
print(model_grayscale)
print(model_10cl)

ResNet(
  (conv1): Sequential(
    (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
    (4): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (5): ReLU(inplace=True)
    (6): Conv2d(32, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
  )
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (act1): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (act1): ReLU(inplace=True)
      (conv2): Co

In [10]:
# 모델의 config 출력

model.default_cfg

{'url': 'https://github.com/rwightman/pytorch-image-models/releases/download/v0.1-weights/resnet50d_ra2-464e36ba.pth',
 'hf_hub_id': 'timm/resnet50d.ra2_in1k',
 'architecture': 'resnet50d',
 'tag': 'ra2_in1k',
 'custom_load': False,
 'input_size': (3, 224, 224),
 'test_input_size': (3, 288, 288),
 'fixed_input_size': False,
 'interpolation': 'bicubic',
 'crop_pct': 0.875,
 'test_crop_pct': 0.95,
 'crop_mode': 'center',
 'mean': (0.485, 0.456, 0.406),
 'std': (0.229, 0.224, 0.225),
 'num_classes': 1000,
 'pool_size': (7, 7),
 'first_conv': 'conv1.0',
 'classifier': 'fc',
 'origin_url': 'https://github.com/huggingface/pytorch-image-models'}

In [11]:
from google.colab import drive
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image
import os
import timm
import pandas as pd
from torchvision.io import read_image

In [12]:
# google drive 연결
drive.mount('/content/drive')

Mounted at /content/drive


In [13]:
# 데이터셋 경로 설정
data_dir = '/content/drive/MyDrive/github-commit-folder/computer-vision-analysis/ISIC2016'

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using {device} device")

Using cuda device


In [14]:
# 데이터셋 클래스 재정의
class ISICDataset(Dataset):
  def __init__(self, csv_file, root_dir, transform=None):
    self.annotations = pd.read_csv(csv_file)
    self.root_dir = root_dir
    self.transform = transform

  def __len__(self):
    return len(self.annotations)

  def __getitem__(self, idx):
    img_name = os.path.join(self.root_dir, self.annotations.iloc[idx, 0] + '.jpg')
    image = read_image(img_name).float() / 255.0
    label = 1 if self.annotations.iloc[idx, 1] == 'malignant' else 0 # 'malignant'와 'benign' 라벨을 정수로 변환

    if self.transform:
      image = self.transform(image)

    return image, torch.tensor(label)

In [15]:
# 이미지 변환 정의
transform = transforms.Compose([
    transforms.Resize((224, 224)), # 이미지 크기 조정
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std= [0.229, 0.224, 0.225]) # 정규화
])

# 데이터셋 및 데이터로더 생성
tr_ds = ISICDataset(csv_file=os.path.join(data_dir,'ISBI2016_ISIC_Part3_Training_GroundTruth.csv'),
                    root_dir=os.path.join(data_dir, 'ISBI2016_ISIC_Part3_Training_Data'),
                    transform=transform)

ts_ds = ISICDataset(csv_file=os.path.join(data_dir, 'ISBI2016_ISIC_Part3_Test_GroundTruth.csv'),
                    root_dir=os.path.join(data_dir, 'ISBI2016_ISIC_Part3_Test_Data'),
                    transform=transform)

tr_loader = DataLoader(tr_ds, batch_size=32, shuffle=True)
ts_loader = DataLoader(ts_ds, batch_size=32)

In [16]:
# declare model
model = timm.create_model('resnet50', pretrained=True, num_classes=2).to(device)

# declare criterion, optimizer
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.001)

In [17]:
# 학습 함수 정의
def train_model(model, dataloader, criterion, optimizer, num_epochs=10, device='cpu'):

  # epoch 수 만큼 아래 내용을 반복
  for epoch in range(num_epochs):

    # 학습에 필요한 초기값 세팅
    model.train()

    # 이번 epoch에서의 loss 값
    running_loss = 0.0

    # 이번 epoch에서의 정답의 수
    correct = 0

    # batch 단위 실행
    total = 0

    # 1번의 batch 단위
    for inputs, labels in dataloader:

      # 이미지와 라벨을 gpu 메모리에 적재
      inputs, labels = inputs.to(device), labels.to(device)

      # 이전에 수행된 Back propagation 초기화
      optimizer.zero_grad()

      # 학습 진행
      outputs = model(inputs)
      loss = criterion(outputs, labels)
      loss.backward()
      optimizer.step()

      # loss 값 계산
      running_loss += loss.item() * inputs.size(0)
      _, predicted = torch.max(outputs, 1)
      total += labels.size(0)
      correct += (predicted == labels).sum().item()

      # 이번 epoch의 평균 loss 계산 및 출력
      epoch_loss = running_loss / len(dataloader.dataset)
      epoch_acc = correct / total

      print(f'Epoch {epoch + 1}/{num_epochs}, Loss: {epoch_loss:.4f}, Accuracy: {epoch_acc:.4f}')

In [18]:
# 모델 학습
train_model(model, tr_loader, criterion, optimizer, num_epochs=10, device=device)

def evaluate_model(model, test_loader, device='cpu'):
  model.eval()
  correct = 0
  total = 0

  with torch.no_grad():

    for images, labels in ts_loader:
      images, labels = images.to(device), labels.to(device)

      outputs = model(images)

      _, predicted = torch.max(outputs, 1)

      total += labels.size(0)
      correct += (predicted == labels).sum().item()
      accuracy = correct / total

      print(f'Test Accuracy:dkdkdkdkkkk {accuracy:.4f}')

      return accuracy

# 테스트 데이터셋을 사용하여 모델 평가
accuracy = evaluate_model(model, ts_loader, device=device)

Epoch 1/10, Loss: 0.0235, Accuracy: 0.7812
Epoch 1/10, Loss: 0.0438, Accuracy: 0.7969
Epoch 1/10, Loss: 0.0619, Accuracy: 0.8021
Epoch 1/10, Loss: 0.0788, Accuracy: 0.8125
Epoch 1/10, Loss: 0.0948, Accuracy: 0.8125
Epoch 1/10, Loss: 0.1122, Accuracy: 0.8073
Epoch 1/10, Loss: 0.1265, Accuracy: 0.8125
Epoch 1/10, Loss: 0.1453, Accuracy: 0.8047
Epoch 1/10, Loss: 0.1615, Accuracy: 0.8056
Epoch 1/10, Loss: 0.1850, Accuracy: 0.7969
Epoch 1/10, Loss: 0.1957, Accuracy: 0.8040
Epoch 1/10, Loss: 0.2036, Accuracy: 0.8151
Epoch 1/10, Loss: 0.2142, Accuracy: 0.8221
Epoch 1/10, Loss: 0.2449, Accuracy: 0.8080
Epoch 1/10, Loss: 0.2649, Accuracy: 0.8042
Epoch 1/10, Loss: 0.2858, Accuracy: 0.8008
Epoch 1/10, Loss: 0.2966, Accuracy: 0.8070
Epoch 1/10, Loss: 0.3065, Accuracy: 0.8108
Epoch 1/10, Loss: 0.3222, Accuracy: 0.8092
Epoch 1/10, Loss: 0.3404, Accuracy: 0.8063
Epoch 1/10, Loss: 0.3510, Accuracy: 0.8095
Epoch 1/10, Loss: 0.3677, Accuracy: 0.8068
Epoch 1/10, Loss: 0.3812, Accuracy: 0.8111
Epoch 1/10,